In [37]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import requests
import json

from shapely.geometry import shape, Point, Polygon
from shapely.ops import unary_union

import rtree as rt

%matplotlib inline
pd.set_option("display.max.columns", None)

## TODOs:
- Based on call with Mo: weighted average of three nearest sensors, figure out AQI, use weekly average data
- Retitle columns to be useful
- Find more complete census tract geojson/one that isn't hosted by latimes is liable to go away at any moment
- How to handle tracts with multiple sensors in them?
- Determine flow:
    - Census tracts won't change, but each sensor can go offline
    - How often to collect PurpleAir data
        - Entire pipeline needs to run at each collection to ensure current/accurate sensors
    - Census tract polygon object gets mapped to a nearest PurpleAir sensor
        - How to make nearest neighbor calculation as efficient as possible
    - How to rebuild geojson/arcgis object from dataframes
    - Where to host processed data?
    - How to package data for dashboard
- Schedule hosted feature layer to be updated:
    - https://learn.arcgis.com/en/projects/schedule-automated-near-real-time-data-updates/

### PurpleAir Data descriptors
##### Determined
- pm = current PM2.5 reading
- pm1 = raw PM1 reading
- pm_10 = raw PM10 reading
- pm_0 = current PM2.5 reading
- pm_1 = 10 minute PM2.5 average
- pm_2 = 30 minute PM2.5 average
- pm_3 = 1 hour PM2.5 average
- pm_4 = 6 hour PM2.5 average
- pm_5 = 24 hour PM2.5 average
- pm_6 = One week PM2.5 average
- p1 = Particles >= 0.3 µm
- p2 = Particles >= 0.5 µm
- p3 = Particles >= 1.0 µm
- p4 = Particles >= 2.5 µm
- p5 = Particles >= 5.0 µm
- p6 = Particles >= 10.0 µm
- flags = Data flagged for unusually high readings
- age = Sensor data age (when data was last received) in minutes
- isOwner = Currently logged in user is the sensor owner
- Adc = The voltage reading on the analog input of the control board

##### Undetermined
- conf
- Icon
- Voc
- Ozone1
- CH

In [38]:
# Returns dataframe of active purpleair sensors contained within LA County
def los_angeles_county_sensors(sensors_df):
    # Set boolean to use LAC shape or LAC bounding box (includes some Orange County and Ventura County)
    use_shape = False
    
    # Initialize rtree index
    idx = rt.index.Index()
    
    # Get LA County geometries in JSON form
    url = 'https://opendata.arcgis.com/datasets/10f1e37c065347e693cf4e8ee753c09b_15.geojson'
    shapes = requests.get(url).json()
    
    la_county_indeces = []
    
    # Union of individual LA County shapes. Speeds up check if sensor is in LAC.
    polygons = [shape(feature['geometry']) for feature in shapes['features']]
    lac_shape = unary_union(polygons)
    lac_bounds = lac_shape.bounds
    
    
    for index, row in sensors_df.iterrows():
        if ~np.isnan(row.Lon) and ~np.isnan(row.Lat):
            
            if use_shape:
                # Build Point object from sensor's Lon and Lat (in that order!) values
                point = Point(row.Lon, row.Lat)

                # Populate rtree index with point bounds
                idx.insert(int(index), point.bounds, point)
            else: 
                if lac_bounds[0] <= row.Lon and row.Lon <= lac_bounds[2] and lac_bounds[1] <= row.Lat and row.Lat <= lac_bounds[3]:
                    la_county_indeces.append(int(index))
    
    if use_shape:
        # get list of fids where bounding boxes intersect
        fids = [i for i in idx.intersection(lac_shape.bounds, objects=True)]
        print(fids[0].object)

        # access the features that those fids reference
        for fid in fids:
            # check the geometries intersect, not just their bboxs
            if lac_shape.contains(fid.object):
                la_county_indeces.append(fid.id)
                    
                
    return sensors_df.loc[la_county_indeces]

In [39]:
# Use 'experimental' data from purpleair
url = 'https://www.purpleair.com/data.json'
data = requests.get(url).json()

sensors_df = pd.DataFrame(data['data'], columns=data['fields'])
sensors_df = sensors_df.set_index('ID')

# Currently only using 1 week PM2.5 average ('pm_6')
cols_to_keep = ['pm_6', 'Lat', 'Lon']
cols_to_drop = [col for col in sensors_df.columns if col not in cols_to_keep]

sensors_df = sensors_df.drop(cols_to_drop, axis=1)

In [40]:
# Get purpleair sensors in LA County
la_county_sensors_df = los_angeles_county_sensors(sensors_df)
la_county_sensors_df['point'] = la_county_sensors_df.apply(lambda x: Point(x['Lon'], x['Lat']), axis=1)
la_county_sensors_df

,pm_6,Lat,Lon,point
ID,,,,
185,30.1,34.072960,-118.264010,POINT (-118.26401 34.07296)
407,24.7,33.842610,-118.339360,POINT (-118.33936 33.84261)
417,23.5,33.716675,-118.309906,POINT (-118.309906 33.716675)
489,28.4,33.795550,-118.260940,POINT (-118.26094 33.79555)
565,16.6,33.709614,-118.287970,POINT (-118.28797 33.709614)
...,...,...,...,...
78739,1.9,34.046070,-118.429245,POINT (-118.429245 34.04607)
78817,2.7,33.703720,-117.717460,POINT (-117.71746 33.70372)
78959,3.4,34.189200,-118.404560,POINT (-118.40456 34.1892)


In [41]:
# Populate rtree index with sensor points
idx = rt.index.Index()
for index, row in la_county_sensors_df.iterrows():
    idx.insert(int(index), row['point'].bounds)

#### Census Tract data is provided by LATimes (not ideal), but is complete.

In [42]:
# Read from url
# test_url = 'http://s3-us-west-2.amazonaws.com/boundaries.latimes.com/archive/1.0/boundary-set/census-tracts-2012.geojson'
# shape_data = requests.get(test_url).json()

# Open saved file
with open('census-tracts-2012.geojson') as f:
    shape_data = json.load(f)

In [43]:
census_tract_df = pd.DataFrame()

census_tract_df['tract'] = [int(feature['properties']['name']) for feature in shape_data['features']]
census_tract_df['coordinates'] = [feature['geometry']['coordinates'] for feature in shape_data['features']]
census_tract_df['shape'] = [shape(feature['geometry']) for feature in shape_data['features']]
census_tract_df.set_index('tract', inplace=True)
census_tract_df

,coordinates,shape
tract,,
6037101110,"[[[[-118.302291, 34.258697], [-118.300787, 34....","(POLYGON ((-118.302291 34.258697, -118.300787 ..."
6037101122,"[[[[-118.303334, 34.273536], [-118.303178, 34....","(POLYGON ((-118.303334 34.273536, -118.303178 ..."
6037101210,"[[[[-118.299451, 34.255978], [-118.285924, 34....","(POLYGON ((-118.299451 34.255978, -118.285924 ..."
6037101220,"[[[[-118.285924, 34.248959], [-118.285924, 34....","(POLYGON ((-118.285924 34.248959, -118.285924 ..."
6037101300,"[[[[-118.272473, 34.232527], [-118.271936, 34....","(POLYGON ((-118.272473 34.232527, -118.271936 ..."
...,...,...
6037980031,"[[[[-118.285303, 33.708598], [-118.283369, 33....","(POLYGON ((-118.285303 33.708598, -118.283369 ..."
6037980033,"[[[[-118.244627, 33.710767], [-118.231803, 33....","(POLYGON ((-118.244627 33.710767, -118.231803 ..."
6037990100,"[[[[-118.951142, 33.996432], [-118.950564, 34....","(POLYGON ((-118.951142 33.996432, -118.950564 ..."


In [44]:
# Use rtree index's nearest function to find indexed point closest to the bounding box of the census tract
# Could easily get the n closest sensors to calculate an average
nearest_sensors = []
for index, row in census_tract_df.iterrows():
    geometry = row['shape']
    
    nearest = list(idx.nearest((geometry.bounds), 3))[:3]
    nearest_sensors.append(nearest)
census_tract_df['nearest_sensor_indices'] = nearest_sensors
census_tract_df

,coordinates,shape,nearest_sensor_indices
tract,,,
6037101110,"[[[[-118.302291, 34.258697], [-118.300787, 34....","(POLYGON ((-118.302291 34.258697, -118.300787 ...","[71813, 67111, 6420]"
6037101122,"[[[[-118.303334, 34.273536], [-118.303178, 34....","(POLYGON ((-118.303334 34.273536, -118.303178 ...","[71813, 67111, 64765]"
6037101210,"[[[[-118.299451, 34.255978], [-118.285924, 34....","(POLYGON ((-118.299451 34.255978, -118.285924 ...","[67111, 71813, 6420]"
6037101220,"[[[[-118.285924, 34.248959], [-118.285924, 34....","(POLYGON ((-118.285924 34.248959, -118.285924 ...","[67111, 63387, 71813]"
6037101300,"[[[[-118.272473, 34.232527], [-118.271936, 34....","(POLYGON ((-118.272473 34.232527, -118.271936 ...","[67111, 63387, 75145]"
...,...,...,...
6037980031,"[[[[-118.285303, 33.708598], [-118.283369, 33....","(POLYGON ((-118.285303 33.708598, -118.283369 ...","[565, 4417, 1974]"
6037980033,"[[[[-118.244627, 33.710767], [-118.231803, 33....","(POLYGON ((-118.244627 33.710767, -118.231803 ...","[22317, 47959, 28529]"
6037990100,"[[[[-118.951142, 33.996432], [-118.950564, 34....","(POLYGON ((-118.951142 33.996432, -118.950564 ...","[38393, 39955, 19991]"


In [45]:
pm_6_cols = []
for index, row in census_tract_df.iterrows():
    closest_vals = []
    
    for sensor_index in row['nearest_sensor_indices']:
        sensor = la_county_sensors_df.loc[sensor_index]
        closest_vals.append(sensor['pm_6'])
    
    pm_6_cols.append(closest_vals)
census_tract_df['pm_6_vals'] = pm_6_cols
census_tract_df

,coordinates,shape,nearest_sensor_indices,pm_6_vals
tract,,,,
6037101110,"[[[[-118.302291, 34.258697], [-118.300787, 34....","(POLYGON ((-118.302291 34.258697, -118.300787 ...","[71813, 67111, 6420]","[18.1, 22.6, 28.7]"
6037101122,"[[[[-118.303334, 34.273536], [-118.303178, 34....","(POLYGON ((-118.303334 34.273536, -118.303178 ...","[71813, 67111, 64765]","[18.1, 22.6, 31.4]"
6037101210,"[[[[-118.299451, 34.255978], [-118.285924, 34....","(POLYGON ((-118.299451 34.255978, -118.285924 ...","[67111, 71813, 6420]","[22.6, 18.1, 28.7]"
6037101220,"[[[[-118.285924, 34.248959], [-118.285924, 34....","(POLYGON ((-118.285924 34.248959, -118.285924 ...","[67111, 63387, 71813]","[22.6, 34.7, 18.1]"
6037101300,"[[[[-118.272473, 34.232527], [-118.271936, 34....","(POLYGON ((-118.272473 34.232527, -118.271936 ...","[67111, 63387, 75145]","[22.6, 34.7, 6.0]"
...,...,...,...,...
6037980031,"[[[[-118.285303, 33.708598], [-118.283369, 33....","(POLYGON ((-118.285303 33.708598, -118.283369 ...","[565, 4417, 1974]","[16.6, 23.2, 24.2]"
6037980033,"[[[[-118.244627, 33.710767], [-118.231803, 33....","(POLYGON ((-118.244627 33.710767, -118.231803 ...","[22317, 47959, 28529]","[26.0, 21.7, 28.5]"
6037990100,"[[[[-118.951142, 33.996432], [-118.950564, 34....","(POLYGON ((-118.951142 33.996432, -118.950564 ...","[38393, 39955, 19991]","[24.8, 20.3, 26.5]"


In [46]:
# Calculate weighted average
census_pm_6 = []

for index, row in census_tract_df.iterrows():
    distances = []
    values = []
    polygon = row['shape']
    for sensor_index in row['nearest_sensor_indices']:
        sensor = la_county_sensors_df.loc[sensor_index]
        distances.append(polygon.boundary.distance(sensor.point))
        values.append(sensor['pm_6'])
    
    weighted_avg = np.average(values, weights=distances)
    
    census_pm_6.append(weighted_avg)
    
census_tract_df['avg_pm_6'] = census_pm_6
census_tract_df

,coordinates,shape,nearest_sensor_indices,pm_6_vals,avg_pm_6
tract,,,,,
6037101110,"[[[[-118.302291, 34.258697], [-118.300787, 34....","(POLYGON ((-118.302291 34.258697, -118.300787 ...","[71813, 67111, 6420]","[18.1, 22.6, 28.7]",23.952856
6037101122,"[[[[-118.303334, 34.273536], [-118.303178, 34....","(POLYGON ((-118.303334 34.273536, -118.303178 ...","[71813, 67111, 64765]","[18.1, 22.6, 31.4]",28.042526
6037101210,"[[[[-118.299451, 34.255978], [-118.285924, 34....","(POLYGON ((-118.299451 34.255978, -118.285924 ...","[67111, 71813, 6420]","[22.6, 18.1, 28.7]",23.432222
6037101220,"[[[[-118.285924, 34.248959], [-118.285924, 34....","(POLYGON ((-118.285924 34.248959, -118.285924 ...","[67111, 63387, 71813]","[22.6, 34.7, 18.1]",23.961380
6037101300,"[[[[-118.272473, 34.232527], [-118.271936, 34....","(POLYGON ((-118.272473 34.232527, -118.271936 ...","[67111, 63387, 75145]","[22.6, 34.7, 6.0]",11.781515
...,...,...,...,...,...
6037980031,"[[[[-118.285303, 33.708598], [-118.283369, 33....","(POLYGON ((-118.285303 33.708598, -118.283369 ...","[565, 4417, 1974]","[16.6, 23.2, 24.2]",22.309035
6037980033,"[[[[-118.244627, 33.710767], [-118.231803, 33....","(POLYGON ((-118.244627 33.710767, -118.231803 ...","[22317, 47959, 28529]","[26.0, 21.7, 28.5]",25.607016
6037990100,"[[[[-118.951142, 33.996432], [-118.950564, 34....","(POLYGON ((-118.951142 33.996432, -118.950564 ...","[38393, 39955, 19991]","[24.8, 20.3, 26.5]",23.786163


In [47]:
def df_to_geojson(df, properties, lat='latitude', lon='longitude'):
    geojson = {'type':'FeatureCollection', 'features':[]}
    for _, row in df.iterrows():
        feature = {'type':'Feature',
                   'properties':{},
                   'geometry':{'type':'Point',
                               'coordinates':[]}}
        feature['geometry']['coordinates'] = [row[lon],row[lat]]
        for prop in properties:
            feature['properties'][prop] = row[prop]
        geojson['features'].append(feature)
    return geojson

In [48]:
geojson = {'type':'FeatureCollection', 'features':[]}
for tract, row in census_tract_df.iterrows():
    feature = {'type': 'Feature',
              'properties': {},
              'geometry': {'type': 'MultiPolygon',
                          'coordinates': []}}
    feature['geometry']['coordinates'] = row['coordinates']
    feature['properties']['tract'] = '0' + str(tract)
    feature['properties']['pm_6_weighted_avg'] = row['avg_pm_6']
    geojson['features'].append(feature)

In [49]:
output_filename = 'data.geojson'
with open(output_filename, 'w') as output_file:
    json.dump(geojson, output_file, indent=2) 

In [50]:
len(geojson['features'])

2346

#### Census Tract data is provided by LATimes (not ideal), but is complete.

In [51]:
# # Iterate through sensors, checking which shape object the sensor's lat/lon falls within
# ind_cen = {}
# for index, row in la_county_sensors_df.iterrows():
#     # Build Point object from sensor's Lon and Lat (in that order!) values
#     point = Point(row.Lon, row.Lat)
    
#     for tract, polygon in polygons.items():
#         if polygon.contains(point):
#             ind_cen[index] = tract

In [52]:
# [i for i in la_county_sensors_df.index if i not in ind_cen.keys()]

#### Different approach by querying fcc's database to get census tract
#### 3.5 minutes to query for 560 locations

In [53]:
# # 3.5 minutes to query for 560 locations
# %%time
# tracts = {}
# for index, row in la_county_sensors_df.iterrows():
#     url = f'https://geo.fcc.gov/api/census/area?lat={row.Lat}&lon={row.Lon}&format=json'
#     results = requests.get(url).json()
#     tracts[index] = results['results'][0]['block_fips'][:11]

In [54]:
# for index, census in ind_cen.items():
#     if census != tracts[index]:
#         print('LATIMES: ', ind_cen[index], 'QUERY: ', tracts[index])